In [1]:
# # # Importing necessary Libraries

import boto
import boto.s3.connection
import xml.etree.ElementTree as et
import pandas as pd
import os
# import logging

access_key = ''
secret_key = ''

#S3 connection establish
conn = boto.connect_s3(aws_access_key_id = '',
                      aws_secret_access_key = '')

In [2]:
Check existing buckets
for bucket in conn.get_all_buckets():
    print(f"Bucket name and date: {bucket.name}\t{bucket.creation_date}")
    

# Creating bucket
bucket_1 = conn.create_bucket('')


# getting bucket object
bucket_2 = conn.get_bucket('')

In [3]:
# Assigning XML file path
path = 'Files/'

# Adding header names
headers = ['Id', 'FullNm', 'ClssfctnTp', 'NtnlCcy', 'CmmdtyDerivInd', 'Issr']
count = 1

'''
1. Going through all .xml files in the system path folder - src/Files/
2. Using XML parser
3. Extracting necessary fields using root.iter() 
4. Extracting text from child node xml blocks
5. Looping through evey parent block
6. Concatenating 2 different DataFrames
7. Saving data to CSV with append option enabled
'''

for files in os.listdir(path):
    if not files.endswith('.xml'):
        continue
        
    files = os.path.join(path, files)
    print(f'{count}. {files}')
    
    # Parsing XML file
    tree = et.parse(files)
    root = tree.getroot()
    
    # Creating row attribute
    rows_1 = []
    
    # looping though all 'FinInstrmGnlAttrbts' parent blocks to extract data from child tags for our columns(tag, attrib)
    for element in root.iter(tag= '{urn:iso:std:iso:20022:tech:xsd:auth.036.001.02}FinInstrmGnlAttrbts'):
        Id = element[0].text
        FullNm = element[1].text
        ClssfctnTp = element[3].text
        NtnlCcy = element[4].text
        CmmdtyDerivInd = element[5].text
        
        # appending values to dictionart
        rows_1.append({"Id" : Id,
                       "FullNm" : FullNm,
                       "ClssfctnTp": ClssfctnTp,
                       "NtnlCcy" : NtnlCcy,
                       "CmmdtyDerivInd" : CmmdtyDerivInd
                    })
        
    # Creating pandas data frame to store values
    df_1 = pd.DataFrame(rows_1)
    
    
    rows_2 = []
    
    # looping through 'Issr' attribute
    for value in root.iter(tag='{urn:iso:std:iso:20022:tech:xsd:auth.036.001.02}Issr'):
        Issr = value.text
        
        # appending values to dictionart
        rows_2.append({"Issr" : Issr})
        
        
    # Creating pandas data frame to store values
    df_2 = pd.DataFrame(rows_2)
    
    # Concatenating 2 different DataFrames
    df = pd.concat([df_1, df_2], axis=1)
    
    # printing dataframe shape
    print(f"DataFrame shape: {df.shape}")
    
    # Printing status message per file
    print(f"{files} - data extraction complete...\n")
    count += 1
    
    # Saving data to Excel CSV format
    df.to_csv('Output/extracted.csv', mode='a', index=False, header=headers)


print('Done!')


1. Files/DLTINS_20210118_01of01.xml
DataFrame shape: (1387, 6)
Files/DLTINS_20210118_01of01.xml - data extraction complete...

2. Files/DLTINS_20210117_01of01.xml
DataFrame shape: (141381, 6)
Files/DLTINS_20210117_01of01.xml - data extraction complete...

3. Files/DLTINS_20210119_01of02.xml
DataFrame shape: (500000, 6)
Files/DLTINS_20210119_01of02.xml - data extraction complete...

4. Files/DLTINS_20210119_02of02.xml
DataFrame shape: (350982, 6)
Files/DLTINS_20210119_02of02.xml - data extraction complete...

Done!


In [4]:
# Removing duplicated rows
dataset = pd.read_csv('Output/extracted.csv')
dataset = dataset.drop(1387)
dataset = dataset.drop(141381)
dataset = dataset.drop(500000)

# verifying duplicate rows removed by checking the shape of our dataset
dataset.shape

/var/folders/kz/m6hkm8w53n9d8ws5s0jvbhd40000gn/T/ipykernel_18099/1780491398.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv('Output/extracted.csv')


(993750, 6)

In [5]:
dataset.to_csv('Output/Final.csv', mode='a', index=False, header=headers)